In [237]:
import pandas as pd
import numpy as np 
import re

In [238]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Main V4.")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Slot Description V4.")

In [239]:
# print slot intent dataframe
df.head()

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN
0,buy,Where can I buy a @parts for my @appliance?\nI...,The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@parts @appliance
1,locate,Where can I locate the @object on the @applian...,The sentence specifies that the user wants to ...,Where can I locate the model number on the mic...,@object @appliance
2,Purpose,What is the purpose of the @feature feature on...,The sentence asks for a purpose of a feature.\...,What is the purpose of the warming center feat...,@feature @appliance
3,cook,What must I do to cook @food with the @applian...,The sentence asks how to prepare a certain typ...,What must I do to cook popcorn with the microw...,@food @appliance
4,cook,How can I @cooking_feature the @food with the ...,The sentence asks how to use a feature to cook...,How can I reheat the chicken with my oven?\nHo...,@food @cooking_feature @appliance


In [240]:
# print slot dataframe
slot_df.head()

,Slot,Description,Examples
0,@parts,A Physical part from an appliance.\nSomething ...,"door, light, storage drawer, exhaust adaptor, ..."
1,@appliance,Any type of appliance. This slot is \nused to ...,microwavawe
2,@object,Anything that is physical but does not belong\...,"door, light, storage-drawer, exhaust-adaptor, ..."
3,@feature,Anything that has settings that can be \nmanip...,"cooking, boiling, melting, surface-cooking, so..."
4,@cooking_feature,A feature specifically used to cook.,"cook, boil, melt, soften, surface cook, airfry..."


In [241]:
# Slot dataframe trimmed so that the Description is removed. 
slot_df_ = slot_df.iloc[:,[0,2]]

In [242]:
# replace the Example column with an array of the values separated by a comma
slot_df_['Examples'] = slot_df_["Examples"].apply(lambda x: x.split(', '))

C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [243]:
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_.Slot, slot_df_.Examples))

In [244]:
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))

In [245]:
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]
# for s in range(len(temp)):
#     for j in range(len(temp[s])):
#         print(temp[s][j])
#     print("-----------------------------------------------")

In [246]:
df

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN
0,buy,"[Where can I buy a @parts for my @appliance?, ...",The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@parts @appliance
1,locate,[Where can I locate the @object on the @applia...,The sentence specifies that the user wants to ...,Where can I locate the model number on the mic...,@object @appliance
2,Purpose,[What is the purpose of the @feature feature o...,The sentence asks for a purpose of a feature.\...,What is the purpose of the warming center feat...,@feature @appliance
3,cook,[What must I do to cook @food with the @applia...,The sentence asks how to prepare a certain typ...,What must I do to cook popcorn with the microw...,@food @appliance
4,cook,[How can I @cooking_feature the @food with the...,The sentence asks how to use a feature to cook...,How can I reheat the chicken with my oven?\nHo...,@food @cooking_feature @appliance
5,Information Code,"[What does @info_code mean on my @appliance?, ...",The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@info_code @appliance
6,Existance,[Is there an @feature feature in the @applianc...,The sentence is asking if a certain function e...,Is there an autocook function in the oven?\nDo...,@feature @appliance
7,Options,[What are the different options of the @featur...,The sentence asks about the different settings...,What are the different options of the Cook fea...,@feature @appliance
8,image,[What does the @object look like in the @appli...,The sentence asks how an object looks like.,What does the Wire racks Look Like in the Micr...,@object @appliance
9,remove,[How do I remove @removeable_parts from the @a...,The sentence asks how to remove something\nfro...,How do I remove burned-on residue from the ove...,@removeable_parts @appliance


In [247]:
#creating the slots for each dictionary
slots = df.iloc[:,4].apply(lambda x: x.split(' '))

In [248]:
target_dic = df.iloc[:,[0,1]]

In [249]:
# for s in range(len(target)):
#     for j in range(len(target[s])):
#         print(target[s][j])
#     print("-----------------------------------------------")

In [250]:
# Print out all the sentences. 
# for s in range(len(temp)): #Intent number
#     for j in range(len(temp[s])): #Sentence in the intent
#         for i in range(len(slot_dict[slots[s][0]])): #Parts slot 1
#             for k in range(len(slot_dict[slots[s][1]])):
#                 x = temp[s][j]
#                 x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
#                 x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
#                 print(x)        
#     print("-----------------------------------------------")

In [252]:
i = 0
for s in range(len(target)): #Intent number
    print(len(slots[s]))
    for j in range(len(target[s])): #Sentence in the intent
        if (len(slots[s]) == 2):
            print("two slots")
            i += two_slots(s,j)    
        elif (len(slots[s]) == 3):
            print("three slots")
            i += three_slots(s,j)
        elif (len(slots[s]) == 1):
            print("one slot")
            i += one_slot(s,j)
    print("-----------------------------------------------")
    
print(i)

2
two slots
Where can I buy a door for my microwavawe?
Where can I buy a light for my microwavawe?
Where can I buy a storage drawer for my microwavawe?
Where can I buy a exhaust adaptor for my microwavawe?
Where can I buy a grease Filter for my microwavawe?
Where can I buy a characol filter for my microwavawe?
Where can I buy a wire racks for my microwavawe?
Where can I buy a air fry tray for my microwavawe?
Where can I buy a glass Tray for my microwavawe?
Where can I buy a grill rack for my microwavawe?
Where can I buy a screws for my microwavawe?
Where can I buy a brackets for my microwavawe?
two slots
I want to get a new door for my microwavawe.
I want to get a new light for my microwavawe.
I want to get a new storage drawer for my microwavawe.
I want to get a new exhaust adaptor for my microwavawe.
I want to get a new grease Filter for my microwavawe.
I want to get a new characol filter for my microwavawe.
I want to get a new wire racks for my microwavawe.
I want to get a new air f

How can I use the microwavawe to microwave-grill the Seasoned?
How can I use the microwavawe to defrost the Seasoned?
How can I use the microwavawe to power defrost the Seasoned?
How can I use the microwavawe to reheat the Seasoned?
How can I use the microwavawe to bake the Seasoned?
How can I use the microwavawe to cook the Frozen Tater tots?
How can I use the microwavawe to boil the Frozen Tater tots?
How can I use the microwavawe to melt the Frozen Tater tots?
How can I use the microwavawe to soften the Frozen Tater tots?
How can I use the microwavawe to surface cook the Frozen Tater tots?
How can I use the microwavawe to airfry the Frozen Tater tots?
How can I use the microwavawe to boil the Frozen Tater tots?
How can I use the microwavawe to warm the Frozen Tater tots?
How can I use the microwavawe to dehydrate the Frozen Tater tots?
How can I use the microwavawe to autocook the Frozen Tater tots?
How can I use the microwavawe to grill the Frozen Tater tots?
How can I use the micr

What is the right settings for soften -ing the Egg Plants in the microwavawe
What is the right settings for surface cook -ing the Egg Plants in the microwavawe
What is the right settings for airfry -ing the Egg Plants in the microwavawe
What is the right settings for boil -ing the Egg Plants in the microwavawe
What is the right settings for warm -ing the Egg Plants in the microwavawe
What is the right settings for dehydrate -ing the Egg Plants in the microwavawe
What is the right settings for autocook -ing the Egg Plants in the microwavawe
What is the right settings for grill -ing the Egg Plants in the microwavawe
What is the right settings for microwave-grill -ing the Egg Plants in the microwavawe
What is the right settings for defrost -ing the Egg Plants in the microwavawe
What is the right settings for power defrost -ing the Egg Plants in the microwavawe
What is the right settings for reheat -ing the Egg Plants in the microwavawe
What is the right settings for bake -ing the Egg Plan

How do I set the speed levels of the echo-mode on my microwavawe?
How do I set the speed levels of the delay-start on my microwavawe?
How do I set the speed levels of the left-rear on my microwavawe?
How do I set the speed levels of the right-rear on my microwavawe?
How do I set the speed levels of the left-front on my microwavawe?
How do I set the speed levels of the right-front on my microwavawe?
How do I set the speed levels of the heating-center on my microwavawe?
How do I set the speed levels of the surface-control on my microwavawe?
How do I set the speed levels of the warming-center on my microwavawe?
How do I set the speed levels of the vent-button on my microwavawe?
-----------------------------------------------
2
two slots
How do I change the power levels with cooking settings on the microwavawe?
How do I change the power levels with boiling settings on the microwavawe?
How do I change the power levels with melting settings on the microwavawe?
How do I change the power level

In [253]:
def two_slots(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
            x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
            p += 1
            print(x)    
    return p

In [254]:
def three_slots(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x = target[s][j]
                x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
                x = re.sub(slots[s][1], slot_dict[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict[slots[s][2]][l], x)
                p += 1
                print(x)   
    return p

In [255]:
def one_slot(s,j):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict[slots[s][0]][i], x)
            p += 1
            print(x)    
    return p

In [280]:
intent_dict = df.QUESTION_LABEL.to_dict()

In [284]:
def print_all(intent_number = len(target), single = False):
    i = 0
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
        print("-----------------------------------------------")
        print("Intent",intent_dict[s],"s:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                #print("two slots")
                i += two_slots(s,j)    
            elif (len(slots[s]) == 3):
                #print("three slots")
                i += three_slots(s,j)
            elif (len(slots[s]) == 1):
                #print("one slot")
                i += one_slot(s,j)
            print("-----------------------------------------------")
    print("-----------------------------------------------")

    print(i)

In [285]:
# print_all(single = False) # Print everything
print_all(2, single=True)

-----------------------------------------------
Intent locate s: 2
Where can I locate the door on the microwavawe?
Where can I locate the light on the microwavawe?
Where can I locate the storage-drawer on the microwavawe?
Where can I locate the exhaust-adaptor on the microwavawe?
Where can I locate the grease-filters on the microwavawe?
Where can I locate the characol-filter on the microwavawe?
Where can I locate the wire-racks on the microwavawe?
Where can I locate the air-fry tray on the microwavawe?
Where can I locate the glass-tray on the microwavawe?
Where can I locate the grill-rack on the microwavawe?
Where can I locate the cooking-button on the microwavawe?
Where can I locate the melting-button on the microwavawe?
Where can I locate the surface-cooking-button on the microwavawe?
Where can I locate the softening-button on the microwavawe?
Where can I locate the airfry-button on the microwavawe?
Where can I locate the boiling-button on the microwavawe?
Where can I locate the keep